# SIR-X

This notebook exemplifies how Open-SIR can be used to fit the SIR-X model by [Maier and Dirk (2020)](https://science.sciencemag.org/content/early/2020/04/07/science.abb4557.full) to existing data and make predictions. The SIR-X model is a standard generalization of the Susceptible-Infectious-Removed (SIR) model, which includes the influence of exogenous factors such as policy changes, lockdown of the whole population and quarantine of the infectious individuals.

The Open-SIR implementation of the SIR-X model will be validated reproducing the parameter fitting published in the [supplementary material](https://science.sciencemag.org/cgi/content/full/science.abb4557/DC1) of the original article published by [Maier and Brockmann (2020)](https://science.sciencemag.org/content/early/2020/04/07/science.abb4557.full). For simplicity, the validation will be performed only for the city of Guangdong, China.

## Import modules

In [ ]:
# Import packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

## Data sourcing

We will source data from the repository of the [John Hopkins University COVID-19 dashboard] (https://coronavirus.jhu.edu/map.html) published formally as a correspondence in [The Lancet](https://www.thelancet.com/journals/laninf/article/PIIS1473-3099(20)30120-1/fulltext#seccestitle10). This time series data contains the number of reported cases $C(t)$ per day for a number of cities.



In [ ]:
# Source data from John Hokpins university reposotiry
# jhu_link = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/who_covid_19_situation_reports/who_covid_19_sit_rep_time_series/who_covid_19_sit_rep_time_series.csv"
jhu_link = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
jhu_df = pd.read_csv(jhu_link)
# Explore the dataset
jhu_df.head(10)

It is observed that the column "Province/States" contains the name of the cities, and since the forth column a time series stamp (or index) is provided to record daily data of reported cases. Additionally, there are many days without recorded data for a number of chinese cities. This won't be an issue for parameter fitting as Open-SIR doesn't require uniform spacement of the observed data.

### Data preparation

In the following lines, the time series for Guangdong reported cases $C(t)$ is extracted from the original dataframe. Thereafter, the columns are converted to a pandas date time index in order to perform further data preparation steps.

In [ ]:
China = jhu_df[jhu_df[jhu_df.columns[1]]=="China"]
city_name = "Guangdong"
city = China[China["Province/State"] == city_name]
city = city.drop(columns=["Province/State", "Country/Region", "Lat","Long"])
time_index = pd.to_datetime(city.columns)
data = city.values
# Visualize the time
ts = pd.Series(data = city.values[0], index = time_index)

Using the function ts.plot() a quick visualization of the dataset is obtained:

In [ ]:
ts.plot()
plt.title('Guangdong COVID-19 cases')
plt.ylabel('$C(t)$: Number of reported cases', size=12)
plt.show()

Data cleaning

In [ ]:
ts_clean = ts.dropna()
# Extract data
ts_fit = ts_clean['2020-01-21':"2020-02-12"]
# Convert index to numeric
ts_num = pd.to_numeric(ts_fit.index)
t0 = ts_num[0]
# Convert datetime to days
t_days = (ts_num-t0)/(10**9*86400)
t_days = t_days.astype(int).values
# t_days is an input for SIR

In [ ]:
# Define the X number
nX = ts_fit.values # Number of infected
N = 104.3e6 # Population size of Guangdong

Exploration of the dataset

In [ ]:
ts_fit.plot(style='ro')
plt.xlabel("Number of infected")
plt.show()

### Setting up SIR and SIR-X models

The population $N$ of the city is a necessary input for the model. In this notebook, this was hardocded, but it can be sourced directly from a web source.

Note that whilst the SIR model estimates directly the number of infected people, $N I(t)$, SIR-X estimates the number of infected people based on the number of tested cases that are in quarantine or in an hospital $N X(t)$

In [ ]:
# These lines are required only if opensir wasn't installed using pip install, or if opensir is running in the pipenv virtual environment
import sys
path_opensir = '../../'
sys.path.append(path_opensir)

# Import SIR and SIRX models
from opensir.models import SIR, SIRX
nX = ts_fit.values # Number of observed infections of the time series
N = 104.3e6 # Population size of Guangdong
params = [0.95, 0.38]
w0 = (N-nX[0], nX[0], 0)

G_sir = SIR()
G_sir.set_params(p=params, initial_conds=w0)
G_sir.fit_input=2
G_sir.fit(t_days, nX)
G_sir.solve(t_days[-1], t_days[-1]+1)
t_SIR = G_sir.fetch()[:,0]
I_SIR = G_sir.fetch()[:,2]

### Try to fit a SIR model to Guangdong data

In [ ]:
ax = plt.axes()
ax.tick_params(axis="both", which="major", labelsize= 14 )
plt.plot(t_SIR, I_SIR)
plt.plot(t_days, nX, 'ro')
plt.show()

The SIR model is clearly not appropriate to fit this data, as it cannot resolve the effect of exogeneous containment efforts such as quarantines or lockdown. We will repeat the process with a SIR-X model.

### Fit SIR-X to Guangdong Data

In [ ]:
g_sirx = SIRX()
params = [6.2/8, 1/8, 0.05, 0.05, 5]
# X_0 can be directly ontained from the statistics
n_x0 = nX[0]            # Number of people tested positive
n_i0 = nX[0]

w0 = (N-n_x0-n_i0, n_i0, 0, n_x0)
g_sirx.set_params(p=params, initial_conds=w0)
# Fit all parameters
fit_index=[False, False, True, True, True]
g_sirx.fit(t_days, nX, fit_index = fit_index)
g_sirx.solve(t_days[-1], t_days[-1]+1)
t_sirx = g_sirx.fetch()[:,0]
inf_sirx = g_sirx.fetch()[:,4]

In [ ]:
plt.figure(figsize=[6,6])
ax = plt.axes()
plt.plot(t_sirx, inf_sirx, 'b-', linewidth=2)
plt.plot(t_SIR, I_SIR,'g-', linewidth=2)
plt.plot(t_days, nX, 'ro')
plt.legend(["SIR-X model fit", "SIR model fit", "Number of reported cases"], fontsize=13)
plt.title("SARS-CoV-2 evolution in Guangdong, China", size=15)
plt.xlabel('Days', fontsize=14)
plt.ylabel('COVID-19 confirmed cases', fontsize=14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.show()

After fitting the parameters, the effective infectious period $T_{I,eff}$ and the effective reproduction rate $R_{0,eff}$ can be obtained from the model properties

$$T_{I,eff} = (\beta + \kappa + \kappa_0)^{-1}$$
$$R_{0,eff} = \alpha T_{I,eff}$$

Aditionally, the Public containment leverage $P$ and the quarantine probability $Q$ can be calculated through:

$$P = \frac{\kappa_0}{\kappa_0 + \kappa}$$
$$Q = \frac{\kappa_0 + \kappa}{\beta + \kappa_0 + \kappa}$$

In [ ]:
print("Effective infectious period T_I_eff =  %.2f days " % g_sirx.t_inf_eff )
print("Effective reproduction rate R_0_eff =  %.2f, Maier and Brockmann = %.2f" % (g_sirx.r0_eff, 3.02))
print("Public containment leverage =  %.2f, Maier and Brockmann = %.2f" % (g_sirx.pcl, 0.75))
print("Quarantine probability =  %.2f, Maier and Brockmann = %.2f" % (g_sirx.q_prob, 0.51))

### Make predictions using `model.predict`

In [ ]:
# Make predictions and visualize
# Obtain the results 14 days after the train data ends
sirx_pred = g_sirx.predict(14)
print('T n_S \t   n_I \tn_R \tn_X')
for i in sirx_pred:
    print(*i.astype(int))

Prepare date time index to plot predictions

In [ ]:
 # Import datetime module from the standard library
import datetime
# Obtain the last day from the data used to train the model
last_time = ts_fit.index[-1] 
# Create a date time range based on the number of rows of the prediction
numdays = sirx_pred.shape[0] 
day_zero = datetime.datetime(last_time.year,last_time.month,last_time.day)
date_list = [day_zero + datetime.timedelta(days=x) for x in range(numdays)]

Plot predictions

In [ ]:
# Extract figure and axes
fig, ax = plt.subplots(figsize = [5,5])
# Create core plot attributes
plt.plot(date_list, sirx_pred[:,2], color = 'red', linewidth =2)
plt.title('Prediction of Guangdong Cases', size=14)
plt.ylabel('Number of infected', size=14)
# Remove trailing space
plt.xlim(date_list[0], date_list[-1])
# Limit the amount of data displayed
ax.xaxis.set_major_locator(plt.MaxNLocator(3))
# Increase the size of the ticks
ax.tick_params(labelsize=12)
plt.show()

### Calculation of predictive confidence intervals

The confidence intervals on the predictions of the SIR-X model can be calculated using a block cross validation. This technique is widely used in Time Series Analysis. In the open-sir API, the function `model.ci_block_cv` calculates the average mean squared error of the predictions, a list of the rolling mean squared errors and the list of parameters which shows how much each parameter changes taking different number of days for making predictions.

The three first parameters are the same as the fit function, while the last two parameters are the `lags` and the `min_sample`. The `lags` parameter indicates how many periods in the future will be forecasted in order to calculate the mean squared error of the model prediction. The `min_sample` parameter indicates the initial number of observations and days that will be taken to perform the block cross validation.

In the following example, `model.ci_block_cv` is used to estimate the average mean squared error of *1-day* predictions taking *6* observations as the starting point of the cross validation. For Guangdong, a `min_sample=6` higher than the default 3 is required to handle well the missing data. This way, both the data on the four first days, and two days after the data starts again, are considered for cross validation.

In [ ]:
# Calculate confidence intervals
mse_avg, mse_list, p_list, pred_data = g_sirx.block_cv(lags = 1, min_sample = 6)

If it is assumed that the residuals distribute normally, then a good estimation of a 95% confidence interval on the one-day prediction of the number of confirmed cases is 

$$\sigma \sim \mathrm{MSE} \rightarrow n_{X,{t+1}} \sim \hat{n}_{X,{t+1}} \pm 2 \sigma$$ 

Where $n_{X,{t+1}}$ is the real number of confirmed cases in the next day, and $\hat{n}_{X,{t+1}}$ is the estimation using the SIR-X model using cross validation. We can use the `PredictionResults` instance `pred_data` functionality to explore the mean-squared errors and the predictions confidence intervals:

In [ ]:
pred_data.print_mse()

The predictive accuracy of the model is quite impressive, even for 9-day predictions. Let's take advantage of the relatively low mean squared error to forecast a 10 days horizon with confidence intervals using `pred_data.plot_predictions(n_days=9)`

In [ ]:
pred_data.plot_predictions(n_days=9)

If it is assumed that the residuals distribute normally, then a good estimation of a 95% confidence interval on the one-day prediction of the number of confirmed cases is 

$$\sigma \sim \mathrm{MSE} \rightarrow n_{X,{t+1}} \sim \hat{n}_{X,{t+1}} \pm 2 \sigma$$ 

Where $n_{X,{t+1}}$ is the real number of confirmed cases in the next day, and $\hat{n}_{X,{t+1}}$ is the estimation using the SIR-X model using cross validation. We use solve to make a 1-day prediction and append the 95% confidence interval.

In [ ]:
# Predict
g_sirx.solve(t_days[-1]+1,t_days[-1]+2)
n_X_tplusone = g_sirx.fetch()[-1,4]
print("Estimation of n_X_{t+1} = %.0f +- %.0f " % (n_X_tplusone, 2*mse_avg[0]) )

In [ ]:
# Transform parameter list into a DataFrame
par_block_cv = pd.DataFrame(p_list)
# Rename dataframe columns based on SIR-X parameter names
par_block_cv.columns = g_sirx.PARAMS
# Add the day. Note that we take the days from min_sample until the end of the array, as days
# 0,1,2 are used for the first sampling in the block cross-validation
par_block_cv['Day'] = t_days[5:]
# Explore formatted dataframe for parametric analysis
par_block_cv.head(len(p_list))

In [ ]:
plt.figure(figsize = [5,5])
ax = plt.axes()
ax.tick_params(axis = "both", which = "major", labelsize = 14 )
plt.plot(mse_list[0],'ro')
plt.xlabel('Number of days used to predict the next day', size = 14)
plt.ylabel('MSE', size = 14)
plt.show()

There is an outlier on day 1, as this is when the missing date starts. A more reliable approach would be to take the last 8 values of the mean squared error to calculate a new average assuming that there will be no more missing data.

#### Variation of fitted parameters

Finally, it is possible to observe how the model parameters change as more days and number of confirmed cases are introduced in the block cross validation. 

It is clear to observe that after day 15 all parameters except kappa begin to converge. Therefore, care must be taken when performing inference over the parameter kappa.

### Long term prediction
Now we can use the model to predict when the peak will occur and what will be the maximum number of infected

In [ ]:
# Predict
plt.figure(figsize=[6,6])
ax = plt.axes()
ax.tick_params(axis="both", which="major", labelsize= 14 )
g_sirx.solve(40,41)
# Plot
plt.plot(g_sirx.fetch()[:,4], 'b-', linewidth=2) # X(t)
plt.plot(g_sirx.fetch()[:,2], 'b--', linewidth=2) # I(t)
plt.xlabel('Day', size=14)
plt.ylabel('Number of people', size=14)
plt.legend(["X(t): Confirmed","I(t) = Infected"], fontsize=13)
plt.title(city_name)
plt.show()

The model was trained with a limited amount of data. It is clear to observe that since the measures took place in Guangdong, at least 6 weeks of quarantine were necessary to control the pandemics. Note that a limitation of this model is that it predicts an equilibrium where the number of infected, denoted by the yellow line in the figure above, is 0 after a short time. In reality, this amount will decrease to a small number.

After the peak of infections is reached, it is necessary to keep the quarantine and effective contact tracing for at least 30 days more.

### Validate using model.plot()

The function `model.plot()` offers a handy way to visualize model fitting and predictions. Custom visualizations can be validated against the `model.plot()` function.

In [ ]:
g_sirx.plot()

Looking at the values of the number of infected and quarantined individuals, as well as the time of the peak, it is clear that both plots are in good agreement, which validates the customized plot created by the code cell below "long term prediction".